In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [21]:
class hybrid(object):
    
    def __init__ (self,user_id,ratings):
        
        self.user_id = user_id
        self.md = pd.read_csv('dataset/processed/FinalData.csv')
        self.ratings = ratings
        self.res=None
        print(ratings)
        print(ratings[(ratings['user_id'] == user_id)][['user_id','book_id', 'rating']])
        print('STARTING CONTENT')
        self.content_rating = self.content_based(self.md,self.ratings,self.user_id)
        print('STARTING POPULARITY')
        self.popularity_rating = self.popularity(self.md)
        print('STARTING COLABORATIVE')
        self.collaborative_rating = self.collaborative(self.ratings, self.user_id)
        print('STARTING HYBRID')
        self.final_hybrid(self.md, self.popularity_rating , self.collaborative_rating, self.content_rating, self.user_id)
        
        
    #Popularity#

    def popularity(self,md):


        fd = pd.read_csv('dataset/processed/AverageRatings.csv')
        fd1 = pd.read_csv('dataset/processed/RatingsCount.csv')
	
        fd[fd['rating'].notnull()]['rating'] = fd[fd['rating'].notnull()]['rating'].astype('float')
        vote_averages= fd[fd['rating'].notnull()]['rating'] 
        C = vote_averages.mean()


        fd1[fd1['rating'].notnull()]['rating'] = fd1[fd1['rating'].notnull()]['rating'].astype('float')
        vote_counts = fd1[fd1['rating'].notnull()]['rating']
        m = len(vote_counts)

        md['ratings_count'] = fd1['rating']
        md['average_rating'] = fd['rating']

        qualified = md[(md['ratings_count'].notnull())][['book_id','title', 'authors', 'ratings_count', 'average_rating']]

        qualified['ratings_count'] = qualified['ratings_count'].astype('float')

        qualified['average_rating'] = qualified['average_rating'].astype('float')

        qualified.shape

        def weighted_rating(x):
            v = x['ratings_count']
            R = x['average_rating']
            return (v/(v+m) * R) + (m/(m+v) * C)

        qualified['popularity_rating'] = qualified.apply(weighted_rating, axis=1)
        pop = qualified[['book_id','popularity_rating']]
        print(qualified.shape)
        print(pop.shape)

        return pop
    ### Collaborative ##

    def collaborative(self,ratings,user_id):

        reader = Reader(rating_scale=(0, 10))
        #ratings.head()

        temp_ratings = ratings



        data = Dataset.load_from_df(temp_ratings[['user_id', 'book_id', 'rating']], reader)
#         data.split(n_folds=2)

        ## Training the data ##
        svd = SVD()
        cross_validate(svd, data, measures=['RMSE', 'MAE'])

        trainset = data.build_full_trainset()

        algo = SVD()
        algo.fit(trainset)

        #svd.train(trainset)
        ## Testing the data ##

        from collections import defaultdict
        testset = trainset.build_anti_testset()
        predictions = algo.test(testset)

        count = 0
     
        for uid, iid, true_r, est, _ in predictions:

             if uid == user_id:
                count = count+1
                temp_ratings.loc[len(temp_ratings)+1]= [uid,iid,est]

        #print("count\n")
        #print(count)
        #print("\n--------here-------\n")	
        #print(temp_ratings)

        cb = temp_ratings[(temp_ratings['user_id'] == user_id)][['book_id', 'rating']]
        #print("\n--------here-------\n")
        #print(cb)
        
        cb = temp_ratings[(temp_ratings['user_id'] == user_id)][['book_id', 'rating']]

        return(cb)


    ##### CONTENT ######

    def content_based(self,md,ratings,user_id):       

        md['book_id'] = md['book_id'].astype('str')
        md['year'] = md['year'].astype('str')
        ratings['book_id'] = ratings['book_id'].astype('str')
        ratings['user_id'] = ratings['user_id'].astype('int')
        ratings['rating'] = ratings['rating'].astype('int')
        md['authors'] = md['authors'].str.replace(' ','')
        md['authors'] = md['authors'].str.lower()
        md['authors'] = md['authors'].str.replace(',',' ')
        md['authors'] = md['authors'].apply(lambda x: [x])
        md['publisher'] = md['publisher'].str.replace(' ','')
        md['publisher'] = md['publisher'].str.lower()
        md['publisher'] = md['publisher'].apply(lambda x: [x])
        md['year'] = md['year'].apply(lambda x: [x])
        md['soup'] = md['authors'] + md['year'] + md['publisher']

        md['soup'] = md['soup'].str.join(' ')
        #print('MD_SOUP',md['soup'])
        count = CountVectorizer(analyzer='word',ngram_range=(1,1),min_df=0, stop_words='english') 
        #how it works? add more attrs
        count_matrix = count.fit_transform(md['soup'].values.astype('U'))
        #print('CM',count_matrix.shape)
        cosine_sim = cosine_similarity(count_matrix, count_matrix)
        #print('CS',cosine_sim.shape)

        def build_user_profiles():
            user_profiles=np.full((278858 ,1000),np.nan)
            for i in range(0,45000):
                u=ratings.iloc[i]['user_id']
                b=ratings.iloc[i]['book_id']
                b2=self.md.index[self.md['book_id'] == b].tolist()[0]
                #b2 is id of book in book attrs table md
                user_profiles[u][b2-1]=ratings.iloc[i]['rating']
                
            return user_profiles

        user_profiles=build_user_profiles()
        print('profiles done')

        def _get_similar_items_to_user_profile(person_id):
            #Computes the cosine similarity between the user profile and all item profiles

            user_ratings = np.empty((1000,1))
            for i in range(0,999):
                book_sim=cosine_sim[i] #how book is similar to all others
                user_sim=user_profiles[person_id] #user all book ratings
                user_sim_bool = [x is not np.nan for x in user_sim]
                user_sim=np.nan_to_num(user_sim)
                non_zero_sim_sum = book_sim.dot(user_sim_bool)
                user_ratings[i]=(book_sim.dot(user_sim))/non_zero_sim_sum
                #print('URi', user_ratings[i], book_sim, user_sim, non_zero_sim_sum)
                #similarities*ratings/similarities sum = avg rating

            return user_ratings

        content_ratings = _get_similar_items_to_user_profile(user_id)
        num = md[['book_id']]
        num1 = pd.DataFrame(data=content_ratings[0:,0:])
        frames = [num, num1]

        content_rating = pd.concat(frames, axis =1,join_axes=[num.index])
        content_rating.columns=['book_id', 'content_rating']
        #print('CR', content_ratings)

        return(content_rating)

    
    def final_hybrid(self,md, popularity_rating , collaborative_rating, content_rating, user_id):

        hyb = md[['book_id']]
        title = md[['book_id','title', 'year']]

        hyb = hyb.merge(title,on = 'book_id')
        hyb = hyb.merge(self.collaborative_rating,on = 'book_id')
        hyb = hyb.merge(self.popularity_rating, on='book_id')
        hyb = hyb.merge(self.content_rating, on='book_id')

        def weighted_rating(x):
            v = x['rating']
            R = x['popularity_rating']
            c = x['content_rating']
            return 0.4*v + 0.2*R + 0.4 * c


        hyb['hyb_rating'] = hyb.apply(weighted_rating, axis=1)
        hyb = hyb.sort_values('hyb_rating', ascending=False).head(999)
        hyb.columns = ['Book ID' , 'Title', 'year', 'Collaborative Rating', 'Popularity Rating' , 'Content Rating', 'Hybrid Rating']
        self.res=hyb
        print(len(hyb['Hybrid Rating']))
        print(hyb)

In [22]:
ratings = pd.read_csv('dataset/processed/FinalRatings.csv')[0:45000]
userId=11676
print('\n----------------Results for'+str(userId)+'-------------------')
h = hybrid(userId,ratings)


----------------Results for11676-------------------
       user_id     book_id  rating
0       276746  0449006522       0
1       276746  055356451X       0
2       276755  0451166892       5
3       276796  0330332775       5
4       276847  0446364193       0
...        ...         ...     ...
44995   247055  0446612545       0
44996   247055  051513290X       0
44997   247055  0553278746       0
44998   247055  0553576925       0
44999   247055  0671004573       0

[45000 rows x 3 columns]
      user_id     book_id  rating
2928    11676  0002005018       8
2929    11676  0060173289       0
2930    11676  0060188731       6
2931    11676  0060915544       0
2932    11676  0060917016       0
...       ...         ...     ...
3233    11676  3257233051      10
3234    11676  3453212150      10
3235    11676  349913358X       0
3236    11676  8433914545       8
3237    11676  9724119378       0

[310 rows x 3 columns]
STARTING CONTENT
profiles done
STARTING POPULARITY
(1000, 6)
(1000, 2

In [18]:
h.res=h.res.rename(columns={"Book ID": "book_id", "Title": "title","Hybrid Rating":"hybrid_rating","Colaborative Rating":"colab_rating", "Content Rating":"content_rating", "Popularity Rating":"pop_rating"})

### USER CHARACTER

In [5]:
finalRatings = pd.read_csv('dataset/processed/FinalRatings.csv')[0:45000]
userRatings=finalRatings[finalRatings.user_id==11676]
finalData=pd.read_csv('dataset/processed/FinalData.csv')

In [8]:
merged = pd.merge(userRatings, finalData, on=['book_id','book_id'])

In [10]:
merged.sort_values(by=['rating'],ascending=False).head(20)

,user_id,book_id,rating,title,authors,year,publisher
208,11676,0553289063,10,An Inconvenient Woman,Dominick Dunne,1991,Bantam Books
24,11676,014023828X,10,The Tortilla Curtain,T. Coraghessan Boyle,1996,Penguin Books
35,11676,0312195516,10,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA
231,11676,0671027387,10,Deception Point,Dan Brown,2002,Pocket
227,11676,0671004573,10,Before I Say Good-Bye,Mary Higgins Clark,2001,Pocket
39,11676,0312963009,10,Neanderthal: A Novel,John Darnton,1997,St. Martin's Press
220,11676,0609605925,10,Angel Falls,KRISTIN HANNAH,2000,Crown
219,11676,0590450875,10,I Spy: A Book of Picture Riddles,Jean Marzollo,1992,Scholastic
216,11676,0553802488,10,The Face,DEAN KOONTZ,2003,Bantam
212,11676,0553578693,10,The Next Accident,LISA GARDNER,2002,Bantam


## RESULTS

### Hybrid

In [19]:
h.res.head(20)

,book_id,title,year,colab_rating,pop_rating,content_rating,hybrid_rating
310,0520011171,Sappho: A New Translation,[1958],5.000000,2.943586,8.000000,5.788717
250,1558744150,Chicken Soup for the Woman's Soul (Chicken Sou...,[1996],10.000000,2.943637,2.493356,5.586070
475,014023828X,The Tortilla Curtain,[1996],10.000000,2.943616,2.276632,5.499376
285,0451526341,Animal Farm,[2004],10.000000,2.943601,2.147604,5.447762
221,0385511612,Bleachers,[2003],10.000000,2.943586,1.982925,5.381887
666,0440122090,Echoes,[1989],10.000000,2.943586,1.953392,5.370074
370,0590450875,I Spy: A Book of Picture Riddles,[1992],10.000000,2.943586,1.905535,5.350931
363,038542471X,The Client,[1993],10.000000,2.943586,1.825094,5.318755
706,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,[1998],10.000000,2.943586,1.809049,5.312337
820,1860198597,How to Draw Celtic Knotwork: A Practical Handbook,[0],3.775505,2.943586,8.000000,5.298919


### Content based

In [20]:
h.res.sort_values(by=['content_rating'],ascending=False).head(20)

,book_id,title,year,colab_rating,pop_rating,content_rating,hybrid_rating
310,0520011171,Sappho: A New Translation,[1958],5.000000,2.943586,8.000000,5.788717
820,1860198597,How to Draw Celtic Knotwork: A Practical Handbook,[0],3.775505,2.943586,8.000000,5.298919
810,0349101779,The Wasp Factory,[0],0.000000,2.943616,7.000000,3.388723
42,067176537X,The Therapeutic Touch: How to Use Your Hands t...,[1979],5.000000,2.943586,6.137034,5.043531
521,0192815318,Cranford (The World's Classics),[1982],3.394737,2.943613,3.889233,3.502310
374,0684180200,NEW SHOE,[1983],2.965345,2.943595,3.861852,3.319598
797,0718144538,Shattered Uk,[0],6.000000,2.943610,3.595918,4.427089
796,0671525794,The Kingdom by the Sea: A Journey Around Great...,[1984],5.000000,2.943610,3.140237,3.844817
366,0449202631,Danger,[1985],3.759491,2.943586,2.918526,3.259924
677,0446364193,Along Came a Spider (Alex Cross Novels),[1993],8.000000,2.943613,2.891564,4.945348
